Say you want to sort a list of numbers but prioritize one group of numbers to come first.
This pattern is useful when you’re rendering a user interface and want important messages
or exceptional events to be displayed before everything else.

A common way to do this is to pass a helper function as the key argument to a list’s sort method. The helper’s return value will be used as the value for sorting each item in
the list. The helper can check whether the given item is in the important group and can
vary the sort key accordingly.



In [3]:
import logging
from pprint import pprint
from sys import stdout as STDOUT


# Example 1
def sort_priority(values, group):
    def helper(x):
        if x in group:
            return (0, x)
        return (1, x)
    values.sort(key=helper)
    
numbers = [8, 3, 1, 2, 5, 4, 7, 6]
group = {2, 3, 5, 7}
sort_priority(numbers, group)
print(numbers)

[2, 3, 5, 7, 1, 4, 6, 8]


There are three reasons why this function operates as expected

1. Python supports closures: functions that refer to variables from the scope in which they were defined. This is why the helper function is able to access the group argument to  sort_priority.
2. Functions are first-class objects in Python, meaning you can refer to them directly, assign them to variables, pass them as arguments to other functions, compare them in expressions and if statements, etc. This is how the sort method can accept a closure function as the key argument.
3. Python has specific rules for comparing tuples. It first compares items in index zero, then index one, then index two, and so on. This is why the return value from the helper closure causes the sort order to have two distinct groups.


The sorted results are correct, but the found result is wrong. Items from group were
definitely found in numbers, but the function returned False. How could this happen?

When you reference a variable in an expression, the Python interpreter will traverse the
scope to resolve the reference in this order:
1. The current function’s scope
2. Any enclosing scopes (like other containing functions)
3. The scope of the module that contains the code (also called the global scope)
4. The built-in scope (that contains functions like len and str)



In [9]:
def sort_priority2(numbers, group):
    found = False
    def helper(x):
        if x in group:
            found = True  # Seems simple
            return (0, x)
        return (1, x)
    numbers.sort(key=helper)
    return found

found = sort_priority2(numbers, group)
print('Found:', found)
print(numbers)



# Example 5
def sort_priority2(numbers, group):
    found = False         # Scope: 'sort_priority2'
    def helper(x):
        if x in group:
            found = True  # Scope: 'helper' -- Bad!
            return (0, x)
        return (1, x)
    numbers.sort(key=helper)
    return found

Found: False
[2, 3, 5, 7, 1, 4, 6, 8]


In Python 3, there is special syntax for getting data out of a closure. The nonlocal
statement is used to indicate that scope traversal should happen upon assignment for a
specific variable name. The only limit is that nonlocal won’t traverse up to the modulelevel
scope (to avoid polluting globals).

In [7]:
def sort_priority3(numbers, group):
    found = False
    def helper(x):
        nonlocal found
        if x in group:
            found = True
            return (0, x)
        return (1, x)
    numbers.sort(key=helper)
    return found

found = sort_priority3(numbers, group)
print('Found:', found)
print(numbers)


Found: True
[2, 3, 5, 7, 1, 4, 6, 8]


The nonlocal statement makes it clear when data is being assigned out of a closure into
another scope. It’s complementary to the global statement, which indicates that a
variable’s assignment should go directly into the module scope.
However, much like the anti-pattern of global variables, I’d caution against using
nonlocal for anything beyond simple functions. The side effects of nonlocal can be
hard to follow. It’s especially hard to understand in long functions where the nonlocal
statements and assignments to associated variables are far apart.
When your usage of nonlocal starts getting complicated, it’s better to wrap your state
in a helper class. Here, I define a class that achieves the same result as the nonlocal
approach. It’s a little longer, but is much easier to read (see Item 23: “Accept Functions for
Simple Interfaces Instead of Classes” for details on the __call__ special method).


In [8]:
class Sorter(object):
    def __init__(self, group):
        self.group = group
        self.found = False

    def __call__(self, x):
        if x in self.group:
            self.found = True
            return (0, x)
        return (1, x)

sorter = Sorter(group)
numbers.sort(key=sorter)
assert sorter.found is True
print('Found:', found)
print(numbers)

Found: True
[2, 3, 5, 7, 1, 4, 6, 8]


* Cosure functions can refer to variables from any of the scopes in which they were defined.
* By default, closures can’t affect enclosing scopes by assigning variables.
* In Python 3, use the nonlocal statement to indicate when a closure can modify a variable in its enclosing scopes.
* In Python 2, use a mutable value (like a single-item list) to work around the lack of the nonlocal statement.
* Avoid using nonlocal statements for anything beyond simple functions.
